In [2]:
import pandas as pd

# Load Merged Dataset
claims_denials_appt = pd.read_excel('/Users/kirsten/claims_denials_appt_merged.xlsx')
claims_denials_appt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11093 entries, 0 to 11092
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   De_ID                           11093 non-null  object        
 1   srvday                          11093 non-null  datetime64[ns]
 2   firstsubmit                     10916 non-null  datetime64[ns]
 3   lastsubmit                      10916 non-null  datetime64[ns]
 4   Total_Charge                    11093 non-null  float64       
 5   proccode                        11093 non-null  object        
 6   icd10claimdiagcode01            11093 non-null  object        
 7   icd10claimdiagcode02            10042 non-null  object        
 8   icd10claimdiagcode03            8760 non-null   object        
 9   icd10claimdiagcode04            7448 non-null   object        
 10  icd10claimdiagcode05            6242 non-null   object        
 11  ic

## Clean Up Variable Names Before Modeling

In [3]:
# Step 1: Dynamic renaming for the 12 ICD diagnosis code columns
icd_renames = {
    f'icd10claimdiagcode{str(i).zfill(2)}': f'ICD10DiagCode_{i}'
    for i in range(1, 13)
}


# Step 2: Manual renaming for additional variables
manual_renames = {
    'patient_age': 'Patient_Age',
    'proccode': 'Procedure_Code',
    'patientsex': 'Patient_Sex',
    'patient_primary_ins_pkg_name': 'Primary_Insurance',
    'patient_primary_ins_pkg_type': 'Primary_Insurance_Type',
    'patient_secondary_ins_pkg_name': 'Secondary_Insurance',
    'patient_secondary_ins_pkg_type': 'Secondary_Insurance_Type',
    'appttype': 'Appt_Type',
    'RAF_score': 'RAF_Score',
    'srvday': 'Appt_Date',
    'firstsubmit': 'Initial_Claim_Submit_Date',
    'lastsubmit': 'Final_Claim_Submit_Date'
}

# Step 4: Combine all rename mappings
all_renames = {**icd_renames, **manual_renames}

# Step 5: Apply the renaming
claims_denials_appt = claims_denials_appt.rename(columns=all_renames)

claims_denials_appt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11093 entries, 0 to 11092
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   De_ID                      11093 non-null  object        
 1   Appt_Date                  11093 non-null  datetime64[ns]
 2   Initial_Claim_Submit_Date  10916 non-null  datetime64[ns]
 3   Final_Claim_Submit_Date    10916 non-null  datetime64[ns]
 4   Total_Charge               11093 non-null  float64       
 5   Procedure_Code             11093 non-null  object        
 6   ICD10DiagCode_1            11093 non-null  object        
 7   ICD10DiagCode_2            10042 non-null  object        
 8   ICD10DiagCode_3            8760 non-null   object        
 9   ICD10DiagCode_4            7448 non-null   object        
 10  ICD10DiagCode_5            6242 non-null   object        
 11  ICD10DiagCode_6            5332 non-null   object        
 12  ICD1

In [4]:
# Drop De_ID (Index variable not needed for modeling)

claims_denials_appt = claims_denials_appt.drop(['De_ID'], axis=1)
claims_denials_appt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11093 entries, 0 to 11092
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Appt_Date                  11093 non-null  datetime64[ns]
 1   Initial_Claim_Submit_Date  10916 non-null  datetime64[ns]
 2   Final_Claim_Submit_Date    10916 non-null  datetime64[ns]
 3   Total_Charge               11093 non-null  float64       
 4   Procedure_Code             11093 non-null  object        
 5   ICD10DiagCode_1            11093 non-null  object        
 6   ICD10DiagCode_2            10042 non-null  object        
 7   ICD10DiagCode_3            8760 non-null   object        
 8   ICD10DiagCode_4            7448 non-null   object        
 9   ICD10DiagCode_5            6242 non-null   object        
 10  ICD10DiagCode_6            5332 non-null   object        
 11  ICD10DiagCode_7            4544 non-null   object        
 12  ICD1

In [5]:
# Impute Denied = 0 (for non-denied claims), then re-check for missing values

# Fill any missing values with 0, then cast to int
claims_denials_appt['Denied'] = claims_denials_appt['Denied'].fillna(0).astype(int)
claims_denials_appt.isnull().sum()

Appt_Date                       0
Initial_Claim_Submit_Date     177
Final_Claim_Submit_Date       177
Total_Charge                    0
Procedure_Code                  0
ICD10DiagCode_1                 0
ICD10DiagCode_2              1051
ICD10DiagCode_3              2333
ICD10DiagCode_4              3645
ICD10DiagCode_5              4851
ICD10DiagCode_6              5761
ICD10DiagCode_7              6549
ICD10DiagCode_8              7290
ICD10DiagCode_9              7957
ICD10DiagCode_10             8545
ICD10DiagCode_11             9050
ICD10DiagCode_12             9556
Primary_Insurance               0
Primary_Insurance_Type          0
Secondary_Insurance             0
Secondary_Insurance_Type        0
Patient_Age                     0
Patient_Sex                     0
Denied                          0
Appt_Type                     212
RAF_Score                     212
dtype: int64

In [8]:
# Variable Transformation and Pre-Processing for Model Building

# Engineer Date Variables
claims_denials_appt['Year_Service'] = claims_denials_appt['Appt_Date'].dt.year
claims_denials_appt['Month_Service'] = claims_denials_appt['Appt_Date'].dt.month
claims_denials_appt['Day_of_Week_Service'] = claims_denials_appt['Appt_Date'].dt.weekday
claims_denials_appt['Initial_Submission_Delay'] = (claims_denials_appt['Initial_Claim_Submit_Date'] - claims_denials_appt['Appt_Date']).dt.days
claims_denials_appt['Final_Submission_Delay'] = (claims_denials_appt['Final_Claim_Submit_Date'] - claims_denials_appt['Appt_Date']).dt.days

#Engineer Fiscal Quarter Variables
def get_fiscal_quarter(date):
    if pd.isnull(date):
        return None
    month = date.month
    if 1 <= month <= 3:
        return 'Q1'
    elif 4 <= month <= 6:
        return 'Q2'
    elif 7 <= month <= 9:
        return 'Q3'
    else:  # 10 <= month <= 12
        return 'Q4'
claims_denials_appt['Quarter_Service'] = claims_denials_appt['Appt_Date'].apply(get_fiscal_quarter)

# Engineer COVID flags

claims_denials_appt['pre_COVID'] = (claims_denials_appt['Year_Service'] < 2020).astype(int)
claims_denials_appt['COVID_era'] = (
    (claims_denials_appt['Year_Service'] >= 2020) & 
    (claims_denials_appt['Year_Service'] <= 2022)
).astype(int)
claims_denials_appt['post_COVID'] = (claims_denials_appt['Year_Service'] > 2022).astype(int)


# To reduce cardinality of appttype variable and per discussion with physician, rename some appointment types as follows:

# Create mapping for specific replacements
appttype_mapping = {
    'ANY 20': 'Follow Up',
    'Established Patient': 'Follow Up',
    'Medicare Annual Wellness': 'Annual Visit',
    'Procedure (cash)': 'Other',
    'Hormone Replacement': 'Other',
    'Lab Work': 'Other',
    'Biofeedback Therapy': 'Other',
    'functional medicine consult': 'Other',
    'COGNITIVE CONSULTATION': 'Other',
    'VACCINATION': 'Other',
    'hair removal': 'Other',
    'cosmetic treatments': 'Other',
    'Preliminary Consultation': 'Other',
    'Membership visit': 'Other'
}
# Apply the mapping
claims_denials_appt['Appt_Type'] = claims_denials_appt['Appt_Type'].replace(appttype_mapping)

# Feature Engineer a new column that gives count of total ICD10 codes included for each claim submission
icd_cols = [
    'ICD10DiagCode_1', 'ICD10DiagCode_2', 'ICD10DiagCode_3',
    'ICD10DiagCode_4', 'ICD10DiagCode_5', 'ICD10DiagCode_6',
    'ICD10DiagCode_7', 'ICD10DiagCode_8', 'ICD10DiagCode_9',
    'ICD10DiagCode_10', 'ICD10DiagCode_11', 'ICD10DiagCode_12'
]
# Count non-null ICD codes per row
claims_denials_appt['ICD10_Code_Count'] = claims_denials_appt[icd_cols].notna().sum(axis=1)

# Feature Engineer combination of Procedure Code & each ICD10 code
import numpy as np
for i in range(1, 13):
    icd_col = f'ICD10DiagCode_{str(i).zfill(1)}'
    new_col = f'Procedure_ICD10_{i}_Combo'
    claims_denials_appt[new_col] = np.where(
        claims_denials_appt[icd_col].notna(),
        claims_denials_appt['Procedure_Code'].astype(str) + '_' + claims_denials_appt[icd_col].astype(str),
        "None"
    )

# Fill missing values of ICD10 columns with "None"
claims_denials_appt[icd_cols] = claims_denials_appt[icd_cols].fillna("None")

# Engineer Procedure Code + Total Charge Pair Variable
claims_denials_appt['Procedure_Charge_Pair'] = (
    claims_denials_appt['Procedure_Code'].astype(str) + '_' + 
    claims_denials_appt['Total_Charge'].round(2).astype(str)
)

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# 1. Handle missing values
claims_denials_appt['Appt_Type'] = claims_denials_appt['Appt_Type'].fillna('None')
claims_denials_appt['RAF_Score'] = claims_denials_appt['RAF_Score'].fillna(claims_denials_appt['RAF_Score'].median())
claims_denials_appt['Initial_Submission_Delay'] = claims_denials_appt['Initial_Submission_Delay'].fillna(claims_denials_appt['Initial_Submission_Delay'].median())
claims_denials_appt['Final_Submission_Delay'] = claims_denials_appt['Final_Submission_Delay'].fillna(claims_denials_appt['Final_Submission_Delay'].median())

# 2. Map Binary categorical mapping
claims_denials_appt['Patient_Sex'] = claims_denials_appt['Patient_Sex'].map({'M': 0, 'F': 1})

# 3. Encode cyclical variables (day of week, month)
claims_denials_appt['Month_Service_sin'] = np.sin(2 * np.pi * claims_denials_appt['Month_Service'] / 12)
claims_denials_appt['Month_Service_cos'] = np.cos(2 * np.pi * claims_denials_appt['Month_Service'] / 12)
claims_denials_appt['Weekday_Service_sin'] = np.sin(2 * np.pi * claims_denials_appt['Day_of_Week_Service'] / 7)
claims_denials_appt['Weekday_Service_cos'] = np.cos(2 * np.pi * claims_denials_appt['Day_of_Week_Service'] / 7)

# 4. One-hot encode low cardinality categorical variables
categorical_cols = ['Primary_Insurance_Type', 'Secondary_Insurance_Type', 'Appt_Type', 'Quarter_Service']
claims_denials_appt = pd.get_dummies(claims_denials_appt, columns=categorical_cols, drop_first=True)

# 5. Scale numeric features (needed for LR and ANN)
from sklearn.preprocessing import StandardScaler
numeric_cols = ['Total_Charge', 'Patient_Age', 'RAF_Score', 'Initial_Submission_Delay',
                'Final_Submission_Delay', 'ICD10_Code_Count']
scaler = StandardScaler()
claims_denials_appt[numeric_cols] = scaler.fit_transform(claims_denials_appt[numeric_cols])

claims_denials_appt.drop(['Appt_Date', 'Initial_Claim_Submit_Date', "Final_Claim_Submit_Date", "Year_Service", "Month_Service", "Day_of_Week_Service"], axis=1, inplace=True)

In [9]:
claims_denials_appt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11093 entries, 0 to 11092
Data columns (total 64 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Total_Charge                                                     11093 non-null  float64
 1   Procedure_Code                                                   11093 non-null  object 
 2   ICD10DiagCode_1                                                  11093 non-null  object 
 3   ICD10DiagCode_2                                                  11093 non-null  object 
 4   ICD10DiagCode_3                                                  11093 non-null  object 
 5   ICD10DiagCode_4                                                  11093 non-null  object 
 6   ICD10DiagCode_5                                                  11093 non-null  object 
 7   ICD10DiagCode_6                         

In [15]:
# Save scaler

joblib.dump(scaler, 'claim_app/model/scaler.pkl')

['claim_app/model/scaler.pkl']

In [ ]:
print("🧪 Final input shape:", claims_denials_appt.shape)
print("🧪 Final columns:", claims_denials_appt.columns.tolist())


In [13]:
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline as ImbPipeline
from category_encoders import TargetEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score)
from sklearn.model_selection import train_test_split, StratifiedKFold

# --- Prepare data ---
target = 'Denied'
diag_cols = [f"ICD10DiagCode_{str(i).zfill(1)}" for i in range(1, 13)]
proc_icd_combo_cols = [f"Procedure_ICD10_{i}_Combo" for i in range(1, 13)]
cat_features = ["Procedure_Code", "Primary_Insurance", 
                "Secondary_Insurance", "Procedure_Charge_Pair"] + diag_cols + proc_icd_combo_cols

# Drop De_ID and un-encoded date variables
X = claims_denials_appt.drop(columns=[target])
y = claims_denials_appt[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    stratify=y, random_state=88)
def find_best_threshold(y_true, y_proba, metric='f1'):
    thresholds = np.linspace(0, 1, 101)
    best_thresh = 0.5
    best_score = 0

    for thresh in thresholds:
        y_pred_thresh = (y_proba >= thresh).astype(int)
        if metric == 'f1':
            score = f1_score(y_true, y_pred_thresh)
        elif metric == 'precision':
            score = precision_score(y_true, y_pred_thresh)
        elif metric == 'recall':
            score = recall_score(y_true, y_pred_thresh)
        else:
            raise ValueError("Metric not supported")

        if score > best_score:
            best_score = score
            best_thresh = thresh

    return best_thresh, best_score


# --- Cross-validation ---
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=88)
results = {}

# Build pipeline
pipe = ImbPipeline(steps=[
    ('encoder', TargetEncoder(cols=cat_features, handle_unknown='value', handle_missing='value')),
    ('smote', SMOTE(random_state=88)),
    ('model', LogisticRegression(C=0.1, random_state=88, max_iter=1000, class_weight='balanced'))
])

# Fit model
pipe.fit(X_train, y_train)

# Predict probabilities
y_proba = pipe.predict_proba(X_test)[:, 1]

# Find best threshold on test set
best_thresh, best_f1 = find_best_threshold(y_test, y_proba, metric='f1')
print(f"\n🔍 Logistic Regression - Best threshold for max F1: {best_thresh:.2f} (F1={best_f1:.3f})")

# Apply threshold
y_pred_thresh = (y_proba >= best_thresh).astype(int)

# Evaluate
results = {
    'AUC': roc_auc_score(y_test, y_proba),
    'Accuracy': accuracy_score(y_test, y_pred_thresh),
    'Precision': precision_score(y_test, y_pred_thresh),
    'Recall': recall_score(y_test, y_pred_thresh),
    'F1': best_f1,
    'Best Threshold': best_thresh
}

# Display
results_df = pd.DataFrame([results], index=["Logistic Regression"])
print("\n📊 Final Test Set Results:\n")
print(results_df.round(4))

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



🔍 Logistic Regression - Best threshold for max F1: 0.34 (F1=0.442)

📊 Final Test Set Results:

                        AUC  Accuracy  Precision  Recall      F1  \
Logistic Regression  0.7575    0.8212     0.3592  0.5756  0.4424   

                     Best Threshold  
Logistic Regression            0.34  


In [14]:
# Save the Model for Flask App

import joblib

# Save the trained pipeline
joblib.dump(pipe, "model.pkl")

['model.pkl']

In [16]:
# After training
joblib.dump(X.columns.tolist(), 'claim_app/model/expected_columns.pkl')


['claim_app/model/expected_columns.pkl']